In [1]:
print('start ch uploads')
#Clickhouse db w/ Goldsky
# https://clickhouse.com/docs/en/integrations/python

import requests as r
import pandas as pd
import clickhouse_connect as cc
import os

import sys
sys.path.append("../helper_functions")
import duneapi_utils as d
import pandas_utils as p
import clickhouse_utils as ch
import csv_utils as cu
import google_bq_utils as bqu
import opstack_metadata_utils as ops
import goldsky_db_utils as gsb
sys.path.pop()

import time

start ch uploads


In [2]:
client = ch.connect_to_clickhouse_db() #Default is OPLabs DB
# client.close()

table_name = 'daily_aggegate_l2_chain_usage_goldsky'

In [3]:
# bqu.delete_bq_table('api_table_uploads',table_name)

In [4]:
days_chunk_size = 3 #7 #30 #90
max_execution_secs = 5000

In [5]:
# chain_mappings_list = [
#     # {'schema_name': 'zora', 'display_name': 'Zora', 'has_blob_fields': True},
#     # {'schema_name': 'pgn', 'display_name': 'Public Goods Network', 'has_blob_fields': True},
#     # {'schema_name': 'base', 'display_name': 'Base', 'has_blob_fields': False},
#     # {'schema_name': 'op', 'display_name': 'OP Mainnet', 'has_blob_fields': True},
#     {'schema_name': 'mode', 'display_name': 'Mode', 'has_blob_fields': True},
#     {'schema_name': 'metal', 'display_name': 'Metal', 'has_blob_fields': True},
#     {'schema_name': 'fraxtal', 'display_name': 'Fraxtal', 'has_blob_fields': True},
#     # {'schema_name': 'bob', 'display_name': 'BOB (Build on Bitcoin)', 'has_blob_fields': False},
#     {'schema_name': 'cyber', 'display_name': 'Cyber', 'has_blob_fields': True},
#     {'schema_name': 'mint', 'display_name': 'Mint', 'has_blob_fields': True},
#     # Add more mappings as needed
# ]
# chain_mappings_dict = {item['schema_name']: item['display_name'] for item in chain_mappings_list}


In [6]:
# Get Chain List
chain_configs = ops.get_superchain_metadata_by_data_source('oplabs') # OPLabs db

# if we want to filter for backfills
# chain_configs = chain_configs[chain_configs['blockchain'].isin(['swan','lisk'])]
# chain_configs = chain_configs[chain_configs['blockchain'] == 'orderly']

chain_configs

,chain_name,display_name,mainnet_chain_id,chain_layer,alignment,da_layer,output_root_layer,gas_token,block_time_sec,public_mainnet_launch_date,op_chain_start,blockchain
0,op,OP Mainnet,10,L2,OP Chain,ethereum,ethereum,ETH,2.0,2021-06-23,2021-06-23,op
1,zora,Zora,7777777,L2,OP Chain,ethereum,ethereum,ETH,2.0,2023-06-21,2023-06-14,zora
2,base,Base,8453,L2,OP Chain,ethereum,ethereum,ETH,2.0,2023-08-09,2023-06-15,base
3,mode,Mode,34443,L2,OP Chain,ethereum,ethereum,ETH,2.0,2024-01-31,2024-01-31,mode
5,lisk,Lisk,1135,L2,OP Chain,ethereum,ethereum,ETH,2.0,2024-05-03,2024-05-03,lisk
6,metal,Metal,1750,L2,OP Chain,ethereum,ethereum,ETH,2.0,2024-03-27,2024-05-09,metal
7,mint,Mint,185,L2,OP Chain,ethereum,ethereum,ETH,2.0,2024-05-13,2024-05-15,mint
8,xterio,Xterio,2702128,L2,OP Chain,altda,ethereum,ETH,2.0,2024-06-11,2024-05-24,xterio
11,kroma,Kroma,255,L2,OP Chain,ethereum,ethereum,ETH,2.0,2023-09-05,2024-05-29,kroma
12,orderly,Orderly,291,L2,OP Chain,celestia,ethereum,ETH,2.0,2023-10-06,2024-07-09,orderly


In [7]:
sql_directory = "inputs/sql/"

query_names = [
        # Must match the file name in inputs/sql
        "ch_template_alltime_chain_activity_v2"
]

In [8]:
# Set the memory limit using the settings parameter
settings = {
    'max_memory_usage': 200000000000  # 200GB in bytes
}


In [9]:
import pandas as pd
from datetime import datetime, timedelta

unified_dfs = []
current_date = datetime.now().date()

for qn in query_names:
    for index, chain in chain_configs.iterrows():
        chain_schema = chain['blockchain']
        display_name = chain['display_name']
        block_time_sec = chain['block_time_sec']
        chain_id = chain['mainnet_chain_id']

        # Get Max Date for this chain
        sql = f'''
        SELECT MAX(dt) AS max_dt
        FROM `api_table_uploads.daily_aggegate_l2_chain_usage_goldsky`
        WHERE (chain_id = '{chain_id}' AND chain_id IS NOT NULL)
                OR (chain = '{display_name}')
        '''
        bq_df = bqu.run_query_to_df(sql)
        query_start_date = bq_df['max_dt'].iloc[0]

        if pd.isna(query_start_date):
            # Get Start Date for the Chain
            print(f'{chain_schema} is new, getting first block date')
            firstblock_sql = f'''SELECT timestamp AS first_dt FROM {chain_schema}_blocks WHERE number = 1 AND is_deleted = 0'''
            firstblock_df = client.query_df(firstblock_sql)
            query_start_date = pd.to_datetime(firstblock_df['first_dt'].iloc[0]).date()
        else:
            print(f'{chain_schema} exists, latest aggregation date')
            query_start_date = query_start_date.date() - timedelta(days=days_chunk_size - 1)

        # Calculate the number of days to process
        days_to_process = (current_date - query_start_date).days
        print(f"{chain_schema} - To process: {days_to_process} days, from {query_start_date} to {current_date} \n")
        # Process data in chunks
        for chunk_start in range(days_to_process, 0, -days_chunk_size):
            chunk_end = max(chunk_start - days_chunk_size, 0)
            
            chunk_start_date = current_date - timedelta(days=chunk_start)
            chunk_end_date = current_date - timedelta(days=chunk_end)
            
            # Read the SQL query from file
            with open(os.path.join(sql_directory, f"{qn}.sql"), "r") as file:
                query = file.read()
            print(f"{qn} - {chain_schema} - Processing {chunk_start_date} to {chunk_end_date}")

            # Pass in Params to the query
            query = query.replace("@chain_db_name@", chain_schema)
            query = query.replace("@start_date@", chunk_start_date.strftime('%Y-%m-%d'))
            query = query.replace("@end_date@", chunk_end_date.strftime('%Y-%m-%d'))
            query = query.replace("@block_time_sec@", str(block_time_sec))
            query = query.replace("@max_execution_secs@", str(max_execution_secs))

            query = gsb.process_goldsky_sql(query)

            # Save the query
            output_folder = os.path.join("outputs", "sql")
            os.makedirs(output_folder, exist_ok=True)
            filename = f"{chain_schema}_query.sql"
            file_path = os.path.join(output_folder, filename)
            with open(file_path, 'w') as file:
                file.write(query)

            # Execute the query
            result_df = client.query_df(query, settings=settings)

            if not result_df.empty:
                result_df['chain_raw'] = result_df['chain']
                result_df['chain'] = display_name
                result_df['chain_id'] = result_df['chain_id'].astype('string')
                bqu.append_and_upsert_df_to_bq_table(result_df, table_name, unique_keys=['dt', 'chain', 'network'])
                unified_dfs.append(result_df)
            else:
                print('dataframe is empty')

    write_df = pd.concat(unified_dfs)
    # BQ Upload
    bqu.append_and_upsert_df_to_bq_table(write_df, table_name, unique_keys=['dt', 'chain', 'network'])
    #CSV Upload
    csv_chain_name = display_name.lower().replace(' ', '_').replace('-', '_')
    write_df.to_csv(f"outputs/chain_data/{qn}_{csv_chain_name}.csv", index=False)

op exists, latest aggregation date
op - To process: 7 days, from 2024-09-06 to 2024-09-13 

ch_template_alltime_chain_activity_v2 - op - Processing 2024-09-06 to 2024-09-13


DatabaseError: :HTTPDriver for https://pdmv9lhojy.us-west-2.aws.clickhouse.cloud:8443 returned response code 500)
 Code: 241. DB::Exception: Memory limit (total) exceeded: would use 108.31 GiB (attempt to allocate chunk of 0 bytes), maximum: 108.00 GiB. OvercommitTracker decision: Query was selected to stop by OvercommitTracker.: While executing AggregatingTransform. (MEMORY_LIMIT_EXCEEDED) (version 24.5.1.22957 (official build))


In [ ]:
# Copy to Dune
print('upload bq to dune')
sql = '''
SELECT *
FROM `api_table_uploads.daily_aggegate_l2_chain_usage_goldsky`
'''
bq_df = bqu.run_query_to_df(sql)

dune_table_name = 'ch_template_alltime_chain_activity'
d.write_dune_api_from_pandas(bq_df, dune_table_name,table_description = dune_table_name)